In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
# to know which folder you are in 
import os
print("Current working directory:", os.getcwd())

In [ ]:
# if needed os.chdir
import os
os.chdir('/home/azureuser/cloudfiles/code/Users/Mohamed.Outahajala/azure-ml-labs/Labs/02')
print("Working directory set to:", os.getcwd())

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import joblib

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('./diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# save the model
output_path = './outputs/model.pkl'
os.makedirs('./outputs', exist_ok=True)  # Ensure the outputs directory exists
joblib.dump(model, output_path)
print(f"Model saved to {output_path}")

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:,1])
print('AUC: ' + str(auc))

In [ ]:
# job creation
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster-dev",
    display_name="diabetes-pythonv2-train_regression-job",
    experiment_name="diabetes-training-regression-experiment",
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
print(returned_job.name + " job submitted successfully.")

In [ ]:
# Step 1
# # Register the model from the job outputs
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Get the job name
job_name = returned_job.name

model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/outputs/model.pkl",
    name="diabetes-logistic-regression",
    description="Logistic regression model for diabetes prediction",
    type=AssetTypes.CUSTOM_MODEL,
    tags={"accuracy": "0.774", "AUC": "0.848"}
)

registered_model = ml_client.models.create_or_update(model)
print(f"✅ Model registered: {registered_model.name} (version: {registered_model.version})")

Step 2 : Create a scoring script
Create a scoring script that will handle inference requests

In [ ]:
%%writefile src/score.py
import json
import joblib
import numpy as np
import os

def init():
    """
    This function is called when the container is initialized/started.
    """
    global model
    # Get the path to the registered model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)
    print("Model loaded successfully")

def run(raw_data):
    """
    This function is called for every invocation of the endpoint.
    """
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result = model.predict(data)
        # Return predictions as JSON
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})

Step 3: Create Environment Configuration
Create a conda dependencies file

In [ ]:
%%writefile src/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy
  - pip
  - scikit-learn
  - scipy
  - pip:
      - azureml-defaults
      - inference-schema[numpy-support]
      - joblib

Step 4: Create a Managed Online Endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
from datetime import datetime

# Create a unique endpoint name
endpoint_name = f"diabetes-endpoint-{datetime.now().strftime('%m%d%H%M%f')}"

# Define the endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint for diabetes prediction",
    auth_mode="key"  # or "aml_token" for Azure AD authentication
)

# Create the endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"✅ Endpoint created: {endpoint_name}")

Step 5: Create a Deployment

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment = ManagedOnlineDeployment(
    name="blue",  # Deployment name
    endpoint_name=endpoint_name,
    model=registered_model.id,
    code_configuration=CodeConfiguration(
        code="./src",
        scoring_script="score.py"
    ),
    environment=f"azureml://registries/azureml/environments/sklearn-1.0/versions/latest",
    instance_type="Standard_DS2_v2",
    instance_count=1
)

# Create the deployment
ml_client.online_deployments.begin_create_or_update(deployment).result()
print(f"✅ Deployment created: blue")

# Set traffic to 100% for this deployment
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("✅ Traffic set to 100% for blue deployment")

In [ ]:
# Step 6: Test the Endpoint
import json

# Sample test data (8 features)
test_data = {
    "data": [
        [2, 180, 74, 24, 21, 23.9, 1.488, 22],  # Sample patient data
        [0, 148, 58, 11, 179, 39.19, 0.160, 45]
    ]
}

# Invoke the endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file=None,
    deployment_name="blue",
    request_body=json.dumps(test_data)
)

print("Prediction result:")
print(response)

In [ ]:
#step 7
# Get the endpoint details
endpoint_details = ml_client.online_endpoints.get(name=endpoint_name)

# Get the scoring URI
scoring_uri = endpoint_details.scoring_uri
print(f"Scoring URI: {scoring_uri}")

# Get the authentication key
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key
print(f"Primary Key: {primary_key}")

In [ ]:
#step 8: Use the Endpoint with REST API
import requests

# Prepare the request
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {primary_key}'
}

test_data = {
    "data": [
        [2, 180, 74, 24, 21, 23.9, 1.488, 22]
    ]
}

# Make the prediction request
response = requests.post(scoring_uri, json=test_data, headers=headers)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")